In [3]:
import sys; sys.path.append('..')
import elastic_rods, sparse_matrices, pickle, scipy, linkage_vis, numpy as np, time
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve
from numpy.linalg import norm
from io_redirection import suppress_stdout
import matplotlib
from matplotlib import pyplot as plt
elastic_rods.set_max_num_tbb_threads(6)

AttributeError: module 'elastic_rods' has no attribute 'set_max_num_tbb_threads'

In [1]:
flatLinkage = pickle.load(open('../data/opt_test_flat.pkl', 'rb'))
deployedLinkage = pickle.load(open('../data/opt_test_deployed.pkl', 'rb'))
perturbation_dir = np.load('../data/opt_test_perturbation_dir.npy')
joint_target_perturbation = np.load('../data/opt_test_joint_target_perturb.npy')

NameError: name 'pickle' is not defined

In [ ]:
eopts = elastic_rods.NewtonOptimizerOptions()
eopts.verbose = True
eopts.niter = 50
params = flatLinkage.getPerSegmentRestLength()

In [ ]:
eopts.gradTol = 1e-13
lopt = elastic_rods.LinkageOptimization(flatLinkage, deployedLinkage, eopts)
lopt.joint_pos_tgt = lopt.joint_pos_tgt + joint_target_perturbation

In [ ]:
lopt.beta = 0
lopt.gamma = 0.0
analytic_gradp_J_target = np.dot(lopt.gradp_J_target(params), perturbation_dir)
analytic_gradp_J = np.dot(lopt.gradp_J(params), perturbation_dir)

def derivative_error(eps = 1e-8, targetOnly = False):
    perturbation = eps * perturbation_dir
    fd, analytic = None, None
    if targetOnly:
        fd = (lopt.J_target(params + perturbation) - lopt.J_target(params - perturbation)) / (2 * eps)
        return (fd - analytic_gradp_J_target) / analytic_gradp_J_target
    else:
        fd = (lopt.J(params + perturbation) - lopt.J(params - perturbation)) / (2 * eps)
        return (fd - analytic_gradp_J) / abs(analytic_gradp_J)

Setting beta = 0 (considering only the elastic energy terms of the objective), finite difference derivatives converge nicely to the analytic gradients, getting ~8 digits of accuracy before roundoff error dominates:

In [ ]:
plt.rcParams['figure.figsize'] = [12, 6]
epsilons = np.power(10, np.linspace(-4,-9, 30))
errors = [derivative_error(eps) for eps in epsilons]
plt.loglog(epsilons, np.abs(errors))
plt.show()

The picture is more complicated for the target-fitting term: while the error is low, it's not obvious that the finite difference approximation is converging.

In [ ]:
epsilons = np.power(10, np.linspace(-4,-9, 100))
errors = [derivative_error(eps, targetOnly=True) for eps in epsilons]
plt.loglog(epsilons, np.abs(errors))
plt.show()

Plotting the target-fitting term value (instead of gradient), we see the numerically evaluated objective is discontinuous and highly sensitive to the gradient tolerance used during the equilibrium solve. This is to be expected: if we don't solve for the equilibrium displacement with enough accuracy, the target-fitting objective won't be computed accurately enough for finite differencing.

In particular, we notice a discontinuity in the scatter plot below, which shows the fitting objective term evaluated at multiple steps along the design parameter perturbation vector. Essentially, this discontinuity can be traced back to the decision on whether another newton step is needed. The discontinuous blue curve is generated by always starting the equilibrium solve from the original design's equilibrium. If we instead continuously update the initial guess for the equilibrium as we step along the perturbation vector (orange curve), the equilibrium is solved quite accurately at each step with just a single newton step (making gradTol irrelevant), and the discontinuity disappears. The discontinuity can also be mitigated by reducing gradTol.

In [ ]:
eopts.gradTol = 1e-11
eopts.verbose = 10
lopt = elastic_rods.LinkageOptimization(flatLinkage, deployedLinkage, eopts)
lopt.joint_pos_tgt = lopt.joint_pos_tgt + joint_target_perturbation

def runTest(epsilons, updateInit = False):
    Jfit = []
    energy = []
    for eps in epsilons:
        if updateInit: lopt.newPt(params + eps * perturbation_dir)
        Jfit.append(lopt.J_target(params + eps * perturbation_dir))
        energy.append(lopt.getLinesearchDeployedLinkage().energy())
    return (epsilons, Jfit, energy)
fixed_init = runTest(np.linspace(0, 1e-6, 50))
updated_init = runTest(np.linspace(0, 1e-6, 50), True)

plt.rcParams['figure.figsize'] = [20, 10]
plt.xlim(min(fixed_init[0]), max(fixed_init[0]))
plt.ylim(min(fixed_init[1]), max(fixed_init[1]))
plt.scatter(fixed_init[0], fixed_init[1], label="fixed_init")
plt.scatter(updated_init[0], updated_init[1], marker=".", label="updated_init")
plt.show()
# plt.xlim(min(fixed_init[0]), max(fixed_init[0]))
# plt.ylim(min(fixed_init[2]), max(fixed_init[2]))
# plt.scatter(fixed_init[0], fixed_init[2], label="fixed_init")
# plt.scatter(updated_init[0], updated_init[2], marker=".", label="updated_init")
# plt.show()